In [5]:
import numpy as np

from factor import *

In [31]:
A = Factor(var=[0],
                     card=[2],
                     val=[0.8, 0.2])
B = Factor(var=[0, 1],
                     card=[2, 2],
                     val=[0.4, 0.55, 0.6, 0.45])

In [26]:
print(A)
print(B)

Factor containing 1 variables
---------------------
| X_0 | Probability |
---------------------
|  0  |         0.8 |
|  1  |         0.2 |
---------------------


Factor containing 2 variables
-------------------------
| X_0 X_1 | Probability |
-------------------------
|  0   0  |         0.4 |
|  1   0  |        0.55 |
|  0   1  |         0.6 |
|  1   1  |        0.45 |
-------------------------




In [48]:
def factor_product(A, B):
    """Compute product of two factors.

    Suppose A = phi(X_1, X_2), B = phi(X_2, X_3), the function should return
    phi(X_1, X_2, X_3)
    """
    if A.is_empty():
        return B
    if B.is_empty():
        return A

    # Create output factor. Variables should be the union between of the
    # variables contained in the two input factors
    out = Factor()
    out.var = np.union1d(A.var, B.var)
    #print(out.var)
    out.card = np.zeros(len(out.var), np.int64)
    # print(out.card)
    mapA = np.argmax(out.var[None, :] == A.var[:, None], axis=-1)
    #print(mapA)
    mapB = np.argmax(out.var[None, :] == B.var[:, None], axis=-1)
    #print(mapB)
    out.card[mapA] = A.card
    out.card[mapB] = B.card
    # print(out.card)
    out.val = np.zeros(np.prod(out.card))
    # print(out.val)
    assignments = out.get_all_assignments()
    # print(assignments)
    idxA = assignment_to_index(assignments[:, mapA], A.card)
    # print(assignments[:, mapA])
    # print(idxA)
    idxB = assignment_to_index(assignments[:, mapB], B.card)
    # print(idxB)
    out.val = A.val[idxA]*B.val[idxB]
    # print(out)
    return out
# factor_product(A,B)

In [89]:
def factor_marginalize(factor, var):
    """Sums over a list of variables.

    Args:
        factor (Factor): Input factor
        var (List): Variables to marginalize out

    Returns:
        out: Factor with variables in 'var' marginalized out.
    """
    out = Factor()

    """ YOUR CODE HERE
    Marginalize out the variables given in var
    """
    out.var = np.setdiff1d(factor.var, var)
    out.card = factor.card[np.where(factor.var != var)[0]]
    # print(factor.card)
    # out.card = factor.card[out.var]
    # print(out.card)
    var = np.array(var)
    # print(var[:, None])
    var_axis = tuple(np.where(factor.var == var)[0])
    merge_axis = tuple(np.where(factor.var != var)[0])
    # print(var_axis)

    # print(out.card)
    # merge_axis = tuple(np.where(factor.var!=var)[0])
    # var_axis = tuple(np.where(factor.var == var)[0])
    # print(merge_axis)
    # print(var_axis)
    # print(factor.val)
    out.val = np.zeros(np.prod(out.card))
    # print(out.val)
    all_assignments = factor.get_all_assignments()
    first_ap_row, indices = np.unique(all_assignments[:, merge_axis], return_inverse=True)
    out.val = np.bincount(indices, weights=factor.val)
    # print(all_assignments)
    # print(sums)
    # print(out.var)
    # print(out.card)
    # print(out.val)
    # result = np.column_stack((first_ap_row, out.val))
    # print(result)
    # print(out)
    return out
factor = Factor(var=[2, 3],
                    card=[2, 3],
                    val=[0.4, 0.35, 0.6, 0.45, 0.0, 0.2])
vars_to_marginalize_out = [2]
output = factor_marginalize(factor, vars_to_marginalize_out)
print(output)


Factor containing 1 variables
---------------------
| X_3 | Probability |
---------------------
|  0  |        0.75 |
|  1  |        1.05 |
|  2  |         0.2 |
---------------------




In [43]:
import copy
import numpy as np
from factor import *
def observe_evidence(factors, evidence=None):
    """Modify a set of factors given some evidence

    Args:
        factors (List[Factor]): List of input factors
        evidence (Dict): Dictionary, where the keys are the observed variables
          and the values are the observed values.

    Returns:
        List of factors after observing evidence
    """
    if evidence is None:
        return factors
    out = copy.deepcopy(factors)

    """ YOUR CODE HERE
    Set the probabilities of assignments which are inconsistent with the
    evidence to zero.
    """
    # 找所有的assignment
    # 找出对应位置的索引
    # 把索引处概率修改为0.
    for cur_factor in out:
        cur_all_assignment = cur_factor.get_all_assignments()
        observe_vars = np.array(list(evidence.keys()))
        observe_vals = np.array(list(evidence.values()))
        # print(observe_vars)
        # print(observe_vals)
        intersection = np.intersect1d(cur_factor.var, observe_vars)
        if not len(intersection)>0:
            # print('yes')
            continue
        # print(intersection)
        # print(observe_vars)
        observe_vals = observe_vals[observe_vars == intersection]
        # print(observe_vals)
        observe_vars = intersection
        observe_vars_idx =  np.argmax(cur_factor.var[None, :] == observe_vars[:, None], axis=-1)
        # print(observe_vars_idx)
        # print(cur_all_assignment[:,observe_vars_idx])
        # print(observe_vars)
        unrelated_assignment_mask = ~np.all(cur_all_assignment[:,observe_vars_idx]==observe_vals,axis=1)
        # print(unrelated_assignment_mask)
        cur_factor.val[unrelated_assignment_mask] = 0.0
        # print(out)

    return out

factors = [Factor(var=[0, 1],
                      card=[2, 3],
                      val=[0.4, 0.35, 0.6, 0.45, 0.0, 0.2]),
               Factor(var=[1, 2],
                      card=[3, 2],
                      val=[0.1, 0.2, 0.3, 0.9, 0.8, 0.7])]
evidence = {0:1}
output = observe_evidence(factors, evidence)

[[0]
 [1]
 [0]
 [1]
 [0]
 [1]]
[0]
[ True False  True False  True False]
[Factor containing 2 variables
-------------------------
| X_0 X_1 | Probability |
-------------------------
|  0   0  |           0 |
|  1   0  |        0.35 |
|  0   1  |           0 |
|  1   1  |        0.45 |
|  0   2  |           0 |
|  1   2  |         0.2 |
-------------------------

, Factor containing 2 variables
-------------------------
| X_1 X_2 | Probability |
-------------------------
|  0   0  |         0.1 |
|  1   0  |         0.2 |
|  2   0  |         0.3 |
|  0   1  |         0.9 |
|  1   1  |         0.8 |
|  2   1  |         0.7 |
-------------------------

]


In [45]:
def factor_sum(A, B):
    """Same as factor_product, but sums instead of multiplies
    """
    if A.is_empty():
        return B
    if B.is_empty():
        return A

    # Create output factor. Variables should be the union between of the
    # variables contained in the two input factors
    out = Factor()
    out.var = np.union1d(A.var, B.var)

    # Compute mapping between the variable ordering between the two factors
    # and the output to set the cardinality
    out.card = np.zeros(len(out.var), np.int64)
    mapA = np.argmax(out.var[None, :] == A.var[:, None], axis=-1)
    mapB = np.argmax(out.var[None, :] == B.var[:, None], axis=-1)
    out.card[mapA] = A.card
    out.card[mapB] = B.card

    # For each assignment in the output, compute which row of the input factors
    # it comes from
    out.val = np.zeros(np.prod(out.card))
    assignments = out.get_all_assignments()
    idxA = assignment_to_index(assignments[:, mapA], A.card)
    idxB = assignment_to_index(assignments[:, mapB], B.card)

    """ YOUR CODE HERE
    You should populate the .val field with the factor sum. The code for this
    should be very similar to the factor_product().
    """
    # print(idxA)
    # print(idxB)
    out.val = A.val[idxA]+B.val[idxB]
    # print(out)

    return out
factor0 = Factor(var=[0],
                     card=[2],
                     val=[0.8, 0.2])
# factor1 contains phi(X_1|X_0)
factor1 = Factor(var=[0, 1],
                 card=[2, 2],
                 val=[0.4, 0.55, 0.6, 0.45])

correct = Factor(var=[0, 1],
                 card=[2, 2],
                 val=[1.2, 0.75, 1.4, 0.65])

output = factor_sum(factor0, factor1)

[0 1 0 1]
[0 1 2 3]
Factor containing 2 variables
-------------------------
| X_0 X_1 | Probability |
-------------------------
|  0   0  |         1.2 |
|  1   0  |        0.75 |
|  0   1  |         1.4 |
|  1   1  |        0.65 |
-------------------------




In [46]:
def factor_max_marginalize(factor, var):
    """Marginalize over a list of variables by taking the max.

    Args:
        factor (Factor): Input factor
        var (List): Variable to marginalize out.

    Returns:
        out: Factor with variables in 'var' marginalized out. The factor's
          .val_argmax field should be a list of dictionary that keep track
          of the maximizing values of the marginalized variables.
          e.g. when out.val_argmax[i][j] = k, this means that
            when assignments of out is index_to_assignment[i],
            variable j has a maximizing value of k.
          See test_lab1.py::test_factor_max_marginalize() for an example.
    """
    out = Factor()

    """ YOUR CODE HERE
    Marginalize out the variables given in var.
    You should make use of val_argmax to keep track of the location with the
    maximum probability.
    """
    # 寻找使得剩下的概率最大的分布，按照键值对的形式
    # 先找剩下的索引
    # 按组找最大值
    return out

factor = Factor(var=[3, 4, 5],
                    card=[2, 2, 3],
                    val=[0.1, 0.9, 0.3, 0.5, 0.4, 0.4, 0.2, 0.1, 0.4, 0.2, 0.7, 0.1])
vars_to_marginalize_out = [3, 5]  # Marginalize out X3, X5, leaving X4

print(factor)
correct = Factor(var=[4],
                 card=[2],
                 val=[0.9, 0.7],
                 val_argmax=[
                     {3: 1, 5: 0},  # When X4==0, the maximizing assignment is X_3=1, X_5=0
                     {3: 0, 5: 2},  # When X4==1, the maximizing assignment is X_3=0, X_5=2
                 ])

output = factor_max_marginalize(factor, vars_to_marginalize_out)

Factor containing 3 variables
-----------------------------
| X_3 X_4 X_5 | Probability |
-----------------------------
|  0   0   0  |         0.1 |
|  1   0   0  |         0.9 |
|  0   1   0  |         0.3 |
|  1   1   0  |         0.5 |
|  0   0   1  |         0.4 |
|  1   0   1  |         0.4 |
|  0   1   1  |         0.2 |
|  1   1   1  |         0.1 |
|  0   0   2  |         0.4 |
|  1   0   2  |         0.2 |
|  0   1   2  |         0.7 |
|  1   1   2  |         0.1 |
-----------------------------




In [54]:
def compute_joint_distribution(factors):
    """Computes the joint distribution defined by a list of given factors

    Args:
        factors (List[Factor]): List of factors

    Returns:
        Factor containing the joint distribution of the input factor list
    """
    joint = Factor()

    """ YOUR CODE HERE
    Compute the joint distribution from the list of factors. You may assume
    that the input factors are valid so no input checking is required.
    """
    # print(len(factors))
    for i in range(len(factors)):
        # print(factors[i])
        if i == 0:
            joint = copy.deepcopy(factors[i])
        else:
            joint = factor_product(joint,factors[i])
    # print(joint)
    return joint

factor0 = Factor(var=[0],
                 card=[2],
                 val=[0.8, 0.2])
# factor1 contains phi(X_1|X_0)
factor1 = Factor(var=[0, 1],
                 card=[2, 2],
                 val=[0.4, 0.55, 0.6, 0.45])
factor2 = Factor(var=[0,2],
                 card=[2,2],val=[0.4,0.5,0.6,0.5])
compute_joint_distribution([factor0,factor1,factor2])

3
Factor containing 1 variables
---------------------
| X_0 | Probability |
---------------------
|  0  |         0.8 |
|  1  |         0.2 |
---------------------


Factor containing 2 variables
-------------------------
| X_0 X_1 | Probability |
-------------------------
|  0   0  |         0.4 |
|  1   0  |        0.55 |
|  0   1  |         0.6 |
|  1   1  |        0.45 |
-------------------------


Factor containing 2 variables
-------------------------
| X_0 X_2 | Probability |
-------------------------
|  0   0  |         0.4 |
|  1   0  |         0.5 |
|  0   1  |         0.6 |
|  1   1  |         0.5 |
-------------------------


Factor containing 3 variables
-----------------------------
| X_0 X_1 X_2 | Probability |
-----------------------------
|  0   0   0  |       0.128 |
|  1   0   0  |       0.055 |
|  0   1   0  |       0.192 |
|  1   1   0  |       0.045 |
|  0   0   1  |       0.192 |
|  1   0   1  |       0.055 |
|  0   1   1  |       0.288 |
|  1   1   1  |       0.

Factor containing 3 variables
-----------------------------
| X_0 X_1 X_2 | Probability |
-----------------------------
|  0   0   0  |       0.128 |
|  1   0   0  |       0.055 |
|  0   1   0  |       0.192 |
|  1   1   0  |       0.045 |
|  0   0   1  |       0.192 |
|  1   0   1  |       0.055 |
|  0   1   1  |       0.288 |
|  1   1   1  |       0.045 |
-----------------------------
